- Remove impact when no player close

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Imports

In [ ]:
import os
import re
import cv2
import time
import json
import torch
import imageio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm
from skimage.transform import resize

from sklearn.metrics import roc_auc_score, f1_score

In [ ]:
from params import *
from utils.plot import plot_confusion_matrix, visualize_preds
from utils.post_processing import post_process_adjacency, post_process_view, post_process_view_2
from utils.metrics import boxes_f1_score, precision_calc, get_boxes_from_df

## Load

### Data

In [ ]:
df_train = pd.read_csv(DATA_PATH + 'df_train.csv')

In [ ]:
# df_train['impact'] += 1
# df_train['impact']

In [ ]:
df_train.dropna(inplace=True)

In [ ]:
frame_impacts = df_train[["image_name", "extended_impact"]].groupby('image_name').max().reset_index()
frame_impacts = frame_impacts.rename(columns={"extended_impact": "frame_has_impact"})
df_train = df_train.merge(frame_impacts, on="image_name")
# df_train = df_train[df_train['frame_has_impact'] == 1]

In [ ]:
folds = pd.read_csv(OUT_DIR + "folds.csv")
df_train = df_train.merge(folds, on="video")

In [ ]:
# Multi-label approach
aux_label = np.zeros((len(df_train), 3))

aux_label[:, 0] = df_train['extended_Helmet']
aux_label[:, 1] = np.clip(
    df_train['extended_Shoulder'] + 
    df_train['extended_shoulder'] + 
    df_train['extended_Body'] + 
    df_train['extended_Hand'], 0, 1)
aux_label[:, 2] = df_train['extended_Ground']

In [ ]:
# Single-label approach
aux_label = np.zeros((len(df_train)))

aux_label[df_train['extended_Helmet'] == 1] = 1
aux_label[df_train['extended_Shoulder'] == 1] = 2
aux_label[df_train['extended_shoulder'] == 1] = 2
aux_label[df_train['extended_Body'] == 1] = 2
aux_label[df_train['extended_Hand'] == 1] = 2
aux_label[df_train['extended_Ground'] == 1] = 3

Counter(aux_label)

### Results

In [ ]:
CP_FOLDERS = [
#     LOG_PATH_CLS + "2020-12-11/37/",  # resnet18
#     LOG_PATH_CLS + "2020-12-12/0/",   # resnet34
#     LOG_PATH_CLS + "2020-12-12/7/",   # efficientnet-b3
#     LOG_PATH_CLS + "2020-12-12/9/",    # efficientnet-b2
#     LOG_PATH_CLS + "2020-12-12/11/",    # efficientnet-b1
#     LOG_PATH_CLS + "2020-12-12/14/",    # efficientnet-b4
    
    LOG_PATH_CLS + "2020-12-18/0/",    # resnet18 + aux
    LOG_PATH_CLS + "2020-12-18/1/",    # efficientnet-b3 + aux
    LOG_PATH_CLS + "2020-12-18/2/",    # efficientnet-b1 + aux
]

In [ ]:
configs = [json.load(open(cp_folder + f'config.json', 'r')) for cp_folder in CP_FOLDERS]

In [ ]:
FOLD = None

In [ ]:
preds = []
pred_aux = []

for cp_folder in CP_FOLDERS:
    if FOLD is None:
        preds.append(np.load(cp_folder + 'pred_oof.npy'))
        pred_aux.append(np.load(cp_folder + 'pred_oof_aux.npy'))
    else:
        preds.append(np.load(cp_folder + f'preds_full_{FOLD}.npy'))
        val_idx = np.load(cp_folder + f'val_idx_{FOLD}.npy')
        pred_aux.append(np.load(cp_folder + f'preds_aux_full_{FOLD}.npy'))
        df_train = df_train.iloc[val_idx].copy()
    
pred_oof_aux = np.mean(pred_aux, 0)
df_train['pred'] = np.mean(preds, 0)

In [ ]:
roc_auc_score(df_train['extended_impact'], df_train['pred'])

In [ ]:
_ = plot_confusion_matrix(
    df_train['pred'] > 0.75, 
    df_train['extended_impact'], 
    display_labels=['0', '1'], 
    figsize=(3, 3)
)

In [ ]:
df_train['pred_aux_helmet'] = pred_oof_aux[:, 1]
df_train['pred_aux_body'] = pred_oof_aux[:, 2]
df_train['pred_aux_ground'] = pred_oof_aux[:, 3]

In [ ]:
impact_types = ["helmet", "body", "ground"]
df_train['predicted_impact_type'] = [impact_types[c] for c in np.argmax(pred_oof_aux[:, 1:], 1)]

In [ ]:
Counter(df_train['predicted_impact_type'])

In [ ]:
impact_dict = {"helmet": 0, "body": 1, "shoulder": 1, "hand":1, "ground": 2}

THRESHOLD_PRED = 0.7
df_truth = df_train[df_train['impact'] == 1]
df_pred = df_train[df_train['pred'] > THRESHOLD_PRED]
df_truth_pred = df_pred[df_pred['impact'] == 1].reset_index(drop=True)
df_impact = df_truth_pred[['impactType', 'predicted_impact_type']].copy()

df_impact['predicted_impact_type'] = df_impact['predicted_impact_type'].apply(lambda x: impact_types.index(x))
df_impact['impactType'] = df_impact['impactType'].apply(lambda x: impact_dict[x.lower()])

plot_confusion_matrix(
    df_impact['predicted_impact_type'], 
    df_impact['impactType'], 
    display_labels=impact_types, 
    figsize=(5, 5)
)

## Per player results

In [ ]:
df_train['player_vid'] = df_train['video'] + "_" + df_train['label']

In [ ]:
p = df_train[["player_vid", "impact", "extended_impact", "pred"]].groupby('player_vid').agg(list)

In [ ]:
for idx in np.random.choice(len(p), 10):
    plt.figure(figsize=(15, 5))
    plt.plot(p['extended_impact'].values[idx])
    plt.plot(p['pred'].values[idx])
    for x, i in enumerate(p['impact'].values[idx]):
        if i:
            plt.axvline(x, c="salmon")
    plt.ylim(-0.1, 1.1)
    plt.title(p.index[idx])
    plt.show()

In [ ]:
players = df_train[["player_vid", "extended_impact", "pred"]].groupby('player_vid').agg(max)

In [ ]:
_ = plot_confusion_matrix(players["pred"] > 0.75, players['extended_impact'], display_labels=['0', '1'], figsize=(5, 5))

# Evaluation

In [ ]:
# Probability thresholding
THRESHOLD_PRED = 0.7

# Adjacency post-processing
THRESHOLD_IOU = 0.35
MAX_DIST = 4
MIN_CLUST_SIZE = 0

# View post-processing
MIN_DIST = 10

# Impact post-processing
MAX_FRAME_DIST = 10

## Default

In [ ]:
df_truth = df_train[df_train['impact'] == 1]
df_pred = df_train[df_train['pred'] > THRESHOLD_PRED]
df_pred_pp = df_pred.copy()
videos = df_truth[cols].groupby("video").agg(list).index

In [ ]:
gt_boxes = get_boxes_from_df(df_truth, videos)
pred_boxes = get_boxes_from_df(df_pred_pp, videos)

In [ ]:
score = boxes_f1_score(pred_boxes, gt_boxes)

print(f' -> CV score is {score:.4f}')

In [ ]:
truth_col = 'impact' # 'impact'

df_truth = df_train[df_train[truth_col] == 1]
df_pred = df_train[df_train['pred'] > THRESHOLD_PRED]

df_fn = df_truth[df_truth['pred'] < THRESHOLD_PRED]
df_tp = df_pred[df_pred[truth_col] == 1]
df_fp = df_pred[df_pred[truth_col] != 1]
df_viz = pd.concat([df_fn, df_tp, df_fp])

In [ ]:
for video in df_viz["video"].unique():
    df_video = df_viz[df_viz["video"] == video]
    for frame in sorted(df_video['frame'].unique()):
        plt.figure(figsize=(16, 8))
        visualize_preds(df_video, video, frame, root=IMG_PATH_F, truth_col=truth_col, threshold_pred=THRESHOLD_PRED)
        plt.axis(False)
        plt.show()
    break

## View post-processing

In [ ]:
# df_pred_pp = post_process_view(
#     df_pred_pp,
#     min_dist=MIN_DIST
# )

In [ ]:
# df_pred_pp = post_process_view_2(
#     df_pred.reset_index(drop=True),
# )

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp, videos)

In [ ]:
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> Post-processed CV score is {score:.4f}')

## Adjacent boxes post-processing

In [ ]:
df_pred_pp = post_process_adjacency(
    df_pred_pp,
    threshold=THRESHOLD_IOU,
    max_dist=MAX_DIST,
    min_clust_size=MIN_CLUST_SIZE,
)

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp, videos)

In [ ]:
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> Post-processed CV score is {score:.4f}')

- `threshold_pred = 0.7`:
 - b3 : `0.3480`
 - 18 + b3 : `0.3602`
 - 18 + b1 + b3 : `0.3628`
 - 18 + b1 + b3 + b4 : `0.3541`

## Helmet post-processing

In [ ]:
def frame_box_dist(b1, b2):
    def get_center(box):
        return (box[0] + box[1] / 2, box[2] + box[3] / 2)
    
    frame_dist = np.abs(b1[0] - b2[0])
    
    x1, y1 = get_center(b1[1:])
    x2, y2 = get_center(b2[1:])
    
    box_dist = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
    return frame_dist, box_dist

In [ ]:
def get_helmet_impact_to_drop(df, max_frame_dist=4):
    df_h = df[df['predicted_impact_type'] == "helmet"]
    
    boxes = df_h[['frame', 'x', 'w', 'y', 'h']].values

    # Compute distances
    distances = np.zeros((len(boxes), len(boxes)))
    for i in range(len(boxes)):
        for j in range(i + 1, len(boxes)):
            frame_dist, box_dist = frame_box_dist(boxes[i], boxes[j])
            
            if frame_dist < max_frame_dist:
                distances[i, j] = - 1 / (box_dist + 1)
                distances[j, i] = - 1 / (box_dist + 1)
    
    # Find pairings
    paired = []
    for i in range(len(boxes)):
        for j in range(i + 1, len(boxes)):
            if i == np.argmin(distances[j]) and j == np.argmin(distances[i]):
                paired += [i, j]

    to_drop = []
    for i, (index, _) in enumerate(df_h.iterrows()):
        if i not in paired:
            to_drop.append(index)
            
    return to_drop

In [ ]:
def post_process_helmet_impact(df, max_frame_dist=4):
    to_drop = []
    for keys in tqdm(df.groupby(['gameKey', 'playID']).size().to_dict().keys()):

        tmp_df = df.query('gameKey == @keys[0] and playID == @keys[1]')
        tmp_to_drop = get_helmet_impact_to_drop(tmp_df, max_frame_dist)

        if len(tmp_to_drop) != len(tmp_df):
            to_drop += tmp_to_drop

    return df.drop(index=to_drop).reset_index(drop=True)

In [ ]:
# df_vid = df_preds[df_preds['video'] == '57583_000082_Endzone.mp4'].copy()
# df_vid['predicted_impact_type'][1] = "body"
# df_vid_pp = post_process_helmet_impact(df_vid, max_frame_dist=MAX_FRAME_DIST)

In [ ]:
# df_pred_pp = post_process_helmet_impact(
#     df_pred_pp,
#     max_frame_dist=MAX_FRAME_DIST,
# )

In [ ]:
pred_boxes_pp = get_boxes_from_df(df_pred_pp, videos)

In [ ]:
score = boxes_f1_score(pred_boxes_pp, gt_boxes)

print(f' -> Post-processed CV score is {score:.4f}')

## Viz

In [ ]:
truth_col = 'impact' # 'impact'

df_truths = df_train[df_train[truth_col] == 1]
df_preds = df_pred_pp.copy()

df_fn = df_truths[df_truths['pred'] < THRESHOLD_PRED]
df_tp = df_preds[df_preds[truth_col] == 1]
df_fp = df_preds[df_preds[truth_col] != 1]
df_viz = pd.concat([df_fn, df_tp, df_fp]).reset_index(drop=True)

In [ ]:
def get_match(df_video):
    cols = ['frame', 'x', 'w', 'y', 'h']
    df_video['match'] = 0
    p_df = df_video[df_video['pred'] > THRESHOLD_PRED].reset_index(drop=True)
    t_df = df_video[df_video['impact'] == 1].reset_index(drop=True)

    p = p_df[cols].values
    t = t_df[cols].values
    p[:, 2] += p[:, 1]
    p[:, 4] += p[:, 3]
    t[:, 2] += t[:, 1]
    t[:, 4] += t[:, 3]
    t = t[:, [0, 1, 3, 2, 4]]
    p = p[:, [0, 1, 3, 2, 4]]

    cost_matix, row_ind, col_ind = precision_calc(t, p, return_assignment=True)
    
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i, j] == 0:
            p_df.loc[j, 'match'] = 1
            t_df.loc[i, 'match'] = 1

    return pd.concat([p_df, t_df], 0).drop_duplicates().sort_values('frame').reset_index(drop=True)

In [ ]:
for video in np.random.choice(df_viz["video"].unique(), 5):
# for video in df_viz["video"].unique()[:1]:
    df_video = df_viz[df_viz["video"] == video].reset_index(drop=True)
    df_matched = get_match(df_video)
    for frame in sorted(df_matched['frame'].unique()):
        plt.figure(figsize=(16, 8))
        visualize_preds(
            df_matched, 
            video, 
            frame, 
            root=IMG_PATH_F, 
            truth_col=truth_col,
        )
        plt.axis(False)
        plt.show()